# Data Sciences Project

In [1]:
#!pip install selenium


import bs4
from bs4 import BeautifulSoup

import pandas as pd
import scipy as sc
import numpy as np
import math


import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import clear_output #library that clears a cell's outputs

import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

## Functions

In [2]:
def addData(arr,data):
    arr.append(data.find_element(By.CLASS_NAME,"value").text)


In [3]:
def addCTBar(arr,data):
    try:
        arr.append(data.get_attribute("ct:value"))
    except:
        arr.append(np.nan)

In [4]:
def addText(arr,data):
    try:
        arr.append(data.text)
    except:
        arr.append(np.nan)

In [5]:
def is_In_List(arr,word):
    for t in range(len(arr)):
        if (arr[t] == word):
            #print(word)
            return 1
    return 0

In [6]:
def has_anime(lst):
    try:
        tst = lst.split(' ')
        if (((tst[0] == "TV") or (tst[0] == "Movie") or (tst[0] == "OVA") or (tst[0] == "ONA") or (tst[0] == "Special")) and ((tst[2] == "Finished") or (tst[2] == "Releasing"))):
            return True
        else: 
            return False
    except:
        return False

In [7]:
def anime_date(link):
    
    drv = webdriver.Chrome()
    drv.get(link)
    time.sleep(1)
    
    data = drv.find_elements(By.CLASS_NAME,"data-set")
    
    for d in data:
        
        dstr = d.find_element(By.CLASS_NAME,"type").text
        #print(dstr)
        
        if ((dstr == "Start Date") or (dstr == "Release Date")):
            txt = d.find_element(By.CLASS_NAME,"value").text
            rtxt = [int(s) for s in txt.split() if s.isdigit()][0]
            return rtxt
    return np.nan
            

## Crawling

In [8]:
PATH = "\chromedriver.exe"
driver = webdriver.Chrome()

driver.get("https://anilist.co/search/manga?format=NOVEL")
driver.maximize_window();

time.sleep(2)

In [9]:
for j in range(1500):
    driver.execute_script("window.scrollTo(0," + str(j * 1000) + ");")
    time.sleep(2)
    
links = driver.find_elements(By.CLASS_NAME,"media-card")
link_list = [elem.find_element(By.CLASS_NAME,"cover").get_attribute('href') for elem in links]

driver.quit()

print(len(link_list))#length of the list

6860


In [22]:
def get_data_into_df(link,tempdrive):
    tempdrive.get(link)
    time.sleep(3)
    
    
    #init dataset element list
    try:
        dataset = tempdrive.find_elements(By.CLASS_NAME,"data-set")
    except:
        dataset = []
    
    
    #init ctbar element list
    try:
        ctbar = tempdrive.find_element(By.CLASS_NAME,"grid-section-wrap").find_elements(By.TAG_NAME,"g")[2].find_elements(By.CLASS_NAME,"ct-bar")#.get_attribute("ct:value")
    except:
        ctbar = []
        
    try:
        ctlabels = tempdrive.find_element(By.CLASS_NAME,"grid-section-wrap").find_element(By.CLASS_NAME,"ct-labels").find_elements(By.XPATH,"//span[contains(@class,'ct-label ct-horizontal ct-end')]")
    except:
        ctlabels = []
    
    #init rankings element list
    try:
        ranku = tempdrive.find_element(By.CLASS_NAME,"rankings").find_elements(By.CLASS_NAME,"rank-text")
    except:
        ranku = []
    
    
    #init tags element list
    try:
        tagspoilclick = tempdrive.find_element(By.CLASS_NAME,"spoiler-toggle")
        tagspoilclick.click()
        taglist = tempdrive.find_elements(By.CSS_SELECTOR,".el-tooltip.name")
    except:
        tagspolclick = []
        taglist = []
    
    
    #init relations element list
    try:
        pcell = tempdrive.find_element(By.CLASS_NAME,"relations").find_elements(By.CLASS_NAME,"media-preview-card")
    except:
        pcell = []
        
        
        
    #help with skipped ctbar nums
    ctbarcount = 0
    
    #check if english name exists
    engFlag = True
    
    
    #dataset element features
    format_ = []
    chap_ = []
    vol_ = []
    status_ = []
    start_ = []
    end_ = []
    avg_score_ = []
    mean_score_ = []
    pop_ = []
    favs_ = []
    source_ = []
    genres_ = []
    name_romaj_ = []
    name_eng_ = []
    name_native_ = []
    name_synon_ = []
    release_year = []
    
    #dataset feature flags
    format_Flag = False
    chap_Flag = False
    vol_Flag = False
    status_Flag = False
    start_Flag = False
    end_Flag = False
    avg_score_Flag = False
    mean_score_Flag = False
    pop_Flag = False
    favs_Flag = False
    source_Flag = False
    genres_Flag = False
    name_romaj_Flag = False
    name_eng_Flag = False
    name_native_Flag = False
    name_synon_Flag = False
    
    
    #ctbar element features
    rating_10 = []
    rating_20 = []
    rating_30 = []
    rating_40 = []
    rating_50 = []
    rating_60 = []
    rating_70 = []
    rating_80 = []
    rating_90 = []
    rating_100 = []
    
    #ctbar element flags
    rating_10_Flag = False
    rating_20_Flag = False
    rating_30_Flag = False
    rating_40_Flag = False
    rating_50_Flag = False
    rating_60_Flag = False
    rating_70_Flag = False
    rating_80_Flag = False
    rating_90_Flag = False
    rating_100_Flag = False
    
    #rankings element features
    rated_rank = []
    popul_rank = []
    ratenum = []
    populnum = []
    
    #relations element features

    cell1_type = []

    cell2_type = []
    
    relations_types = []
    
    has_anime_ = []
    anime_Flag = False
    
    anime_rls_date = []

    #Genres
    gen_one = []
    gen_two = []
    gen_thr = []
    
    #Tags
    tag_one = []
    tag_two = []
    tag_thr = []
    
    #Weird feature but let's try
    name_word_count = []
    
    #Gotta save links
    links_ = []

    
    
    #Check and Handle appending of dataset features
    
    for p in dataset:
        
        testStr = p.find_element(By.CLASS_NAME,"type").text
        
        if ((testStr == "Format") and (format_Flag == False)):
            format_Flag = True
            try:
                addData(format_,p)
            except:
                format_.append(np.nan)
        
        if ((testStr == "Chapters") and (chap_Flag == False)):
            chap_Flag = True
            try:
                addData(chap_,p)
            except:
                chap_.append(np.nan)
        
        if ((testStr == "Volumes") and (vol_Flag == False)):
            vol_Flag = True
            try:
                addData(vol_,p)
            except:
                vol_.append(np.nan)
            
        if ((testStr == "Status") and (status_Flag == False)):
            status_Flag = True
            try:
                addData(status_,p)
            except:
                status_.append(np.nan)
        
        if (((testStr == "Start Date") or (testStr == "Release Date")) and (start_Flag == False)):
            start_Flag = True
            try:
                addData(start_,p)
            except:
                start_.append(np.nan)
        
        if ((testStr == "End Date") and (end_Flag == False)):
            end_Flag = True
            try:
                addData(end_,p)
            except:
                end_.append('TBD')
            
        if ((testStr == "Average Score") and (avg_score_Flag == False)):
            avg_score_Flag = True
            try:
                avg_score_temp = []
                addData(avg_score_temp,p)
                avg_score_.append(avg_score_temp[0].split('%')[0])
            except:
                avg_score_.append(np.nan)
            
        if ((testStr == "Mean Score") and (mean_score_Flag == False)):
            mean_score_Flag = True
            try:
                mean_score_temp = []
                addData(mean_score_temp,p)
                mean_score_.append(mean_score_temp[0].split('%')[0])
            except:
                mean_score_.append(np.nan)
                
        if ((testStr == "Popularity") and (pop_Flag == False)):
            pop_Flag = True
            try:    
                addData(pop_,p)
            except:
                pop_.append(np.nan)
            
        if ((testStr == "Favorites") and (favs_Flag == False)):
            favs_Flag = True
            try:
                addData(favs_,p)
            except:
                favs_.append(np.nan)
            
        if ((testStr == "Source") & (source_Flag == False)):
            source_Flag = True
            try:
                addData(source_,p)
            except:
                source_.append(np.nan)
            
        if ((testStr == "Genres") and (genres_Flag == False)):
            genres_Flag = True
            try:
                addData(genres_,p)
            except:
                genres_.append(np.nan)
            
        if ((testStr == "Romaji") and (name_romaj_Flag == False)):
            name_romaj_Flag = True
            try:
                addData(name_romaj_,p)
            except:
                name_romaj_.append(np.nan)
        
        if ((testStr == "English") and (name_eng_Flag == False)):
            name_eng_Flag = True
            try:
                addData(name_eng_,p)
            except:
                name_eng_.append(np.nan)
            
        if ((testStr == "Native") and (name_native_Flag == False)):
            name_native_Flag = True
            try:
                addData(name_native_,p)
            except:
                name_native_.append(np.nan)
            
        if ((testStr == "Synonyms") and (name_synon_Flag == False)):
            name_synon_Flag = True
            try:
                addData(name_synon_,p)
            except:
                name_synon_.append(np.nan)
                
                
    #If any feature was missing from an entry, append NaN  
    if (format_Flag == False):  
        format_.append(np.nan)
        
    if (chap_Flag == False):  
        chap_.append(np.nan)
        
    if (vol_Flag == False):  
        vol_.append(np.nan)

    if (status_Flag == False):  
        status_.append(np.nan)
    
    if (start_Flag == False):  
        start_.append(np.nan)
    
    if (end_Flag == False):  
        end_.append(np.nan)
    
    if (avg_score_Flag == False):  
        avg_score_.append(np.nan)
    
    if (mean_score_Flag == False):  
        mean_score_.append(np.nan)
    
    if (pop_Flag == False):  
        pop_.append(np.nan)
    
    if (favs_Flag == False):  
        favs_.append(np.nan)
    
    if (source_Flag == False):  
        source_.append(np.nan)
    
    if (genres_Flag == False):  
        genres_.append(np.nan)
    
    if (name_romaj_Flag == False):  
        name_romaj_.append(np.nan)
    
    if (name_eng_Flag == False):  
        name_eng_.append(np.nan)
    
    if (name_native_Flag == False):  
        name_native_.append(np.nan)

    if (name_synon_Flag == False):  
        name_synon_.append(np.nan)
    
    
    
    
    if (start_Flag == True):
        yeartext = start_[0]
        release_year.append([int(s) for s in yeartext.split() if s.isdigit()][0])
    else:
        release_year.append(np.nan)
    
    
    
    
    
    
    
    #Handle appending of ctbar features
    for t in range(len(ctbar)):
        
        ctstr = ctlabels[t].get_attribute('innerHTML')
        
        if ((ctstr == '10') and (rating_10_Flag == False)):
            rating_10_Flag = True
            addCTBar(rating_10,ctbar[t])

                
        if ((ctstr == '20') and (rating_20_Flag == False)):
            rating_20_Flag = True
            addCTBar(rating_20,ctbar[t])

                
        if ((ctstr == '30') and (rating_30_Flag == False)):
            rating_30_Flag = True
            addCTBar(rating_30,ctbar[t])

                
        if ((ctstr == '40') and (rating_40_Flag == False)):
            rating_40_Flag = True
            addCTBar(rating_40,ctbar[t])

                
        if ((ctstr == '50') and (rating_50_Flag == False)):
            rating_50_Flag = True
            addCTBar(rating_50,ctbar[t])

                
        if ((ctstr == '60') and (rating_60_Flag == False)):
            rating_60_Flag = True
            addCTBar(rating_60,ctbar[t])
 
                
        if ((ctstr == '70') and (rating_70_Flag == False)):
            rating_70_Flag = True
            addCTBar(rating_70,ctbar[t])

                
        if ((ctstr == '80') and (rating_80_Flag == False)):
            rating_80_Flag = True
            addCTBar(rating_80,ctbar[t])

              
        if ((ctstr == '90') and (rating_90_Flag == False)):
            rating_90_Flag = True
            addCTBar(rating_90,ctbar[t])

              
        if ((ctstr == '100') and (rating_100_Flag == False)):
            rating_100_Flag = True
            addCTBar(rating_100,ctbar[t])
    

                    
                    
                    
    if (rating_10_Flag == False):
        rating_10.append(np.nan)
   
    if (rating_20_Flag == False):
        rating_20.append(np.nan)
    
    if (rating_30_Flag == False):
        rating_30.append(np.nan)
    
    if (rating_40_Flag == False):
        rating_40.append(np.nan)
    
    if (rating_50_Flag == False):
        rating_50.append(np.nan)
    
    if (rating_60_Flag == False):
        rating_60.append(np.nan)
    
    if (rating_70_Flag == False):
        rating_70.append(np.nan)
    
    if (rating_80_Flag == False):
        rating_80.append(np.nan)
    
    if (rating_90_Flag == False):
        rating_90.append(np.nan)
    
    if (rating_100_Flag == False):
        rating_100.append(np.nan)


    
    #Handle appending of rankings features
    try:
        addText(rated_rank,ranku[0])
    except:
        rated_rank.append(np.nan)
        
    try:
        addText(popul_rank,ranku[1])
    except:
        popul_rank.append(np.nan)
        
    if (rated_rank[0] is not np.nan):
        rstr = rated_rank[0].split('#')
        ratenum.append([int(sr) for sr in rstr[1].split() if sr.isdigit()][0])
    else:
        ratenum.append(np.nan)
        
    if (popul_rank[0] is not np.nan):
        pstr = popul_rank[0].split('#')
        populnum.append([int(sp) for sp in pstr[1].split() if sp.isdigit()][0])
    else:
        populnum.append(np.nan)
        
    
    #Handle appending of top 3 tags features
    
    try:
        tag_one.append(taglist[0].text)
    except:
        tag_one.append(np.nan)
        
    try:
        tag_two.append(taglist[1].text)
    except:
        tag_two.append(np.nan)
        
    try:
        tag_thr.append(taglist[2].text)
    except:
        tag_thr.append(np.nan)
        
    #Handle appending of top 3 genres features
    
    try:
        genArr = genres_[0].split('\n')
    except:
        genArr = []
        
    try:
        gen_one.append(genArr[0])
    except:
        gen_one.append(np.nan)
    
    try:    
        gen_two.append(genArr[1])
    except:
        gen_two.append(np.nan)
    
    try:    
        gen_thr.append(genArr[2])
    except:
        gen_thr.append(np.nan)
    
    try:
        name_word_count.append(len(name_romaj_[0].split(' ')))
    except:
        name_word_count.append(np.nan)
    
    #Handle appending of relations features

    try:
        cell1_type.append(pcell[0].find_element(By.CLASS_NAME,"content").find_element(By.CLASS_NAME,"info").text)
    except:
        cell1_type.append(np.nan)

    try:
        cell2_type.append(pcell[1].find_element(By.CLASS_NAME,"content").find_element(By.CLASS_NAME,"info").text)
    except:
        cell2_type.append(np.nan)
    
    
    
    
    
    #Handle determining if there is an anime, and what was its release date
    try:
        for r in pcell:
            relations_types.append(r.find_element(By.CLASS_NAME,"content").find_element(By.CLASS_NAME,"info").text)
              
    except:
        relations_types.append(np.nan)
            
    #Check if this has an anime
    try:
        if (relations_types[0] is not np.nan):
            for rt in range(0,len(relations_types)):
                if (anime_Flag == False):
                    anime_Flag = has_anime(relations_types[rt])
                    if (anime_Flag == True):
                        rts = pcell[rt]
                
            if (anime_Flag == True):
                anime_rls_date.append(anime_date(rts.find_element(By.CLASS_NAME,"title").get_attribute('href')))
            else:
                anime_rls_date.append(np.nan)
    except:
        anime_rls_date.append(np.nan)
        
    
    has_anime_.append(anime_Flag)
        

    
    
    links_.append(link)
        
    data = {"Name Romaji":name_romaj_,
            "Name Native":name_native_,
            "Name English":name_eng_,
            "Synonyms":name_synon_,
            "Format":format_,
            "Chapter Count":chap_,
            "Volume Count":vol_,
            "Status":status_,
            "Start Date":start_,
            "Start Year":release_year,
            "End Date":end_,
            "Average Score":avg_score_,
            "Mean Score":mean_score_,
            "Popularity":pop_,
            "Favorites":favs_,
            "Source":source_,
            
            "Genre_One":gen_one,
            "Genre_Two":gen_two,
            "Genre_Three":gen_thr,
            
            "Tag_One":tag_one,
            "Tag_Two":tag_two,
            "Tag_Three":tag_thr,
            
            "Rating 10":rating_10,
            "Rating 20":rating_20,
            "Rating 30":rating_30,
            "Rating 40":rating_40,
            "Rating 50":rating_50,
            "Rating 60":rating_60,
            "Rating 70":rating_70,
            "Rating 80":rating_80,
            "Rating 90":rating_90,
            "Rating 100":rating_100,
            
            "Rating Rank":rated_rank,
            "Rating Number":ratenum,
            "Popularity Rank":popul_rank,
            "Popularity Number":populnum,
            
            "Related 1 Type":cell1_type,
            "Related 2 Type":cell2_type,
            "Has Anime":has_anime_,
            "Anime Release Date":anime_rls_date,
            
            "Name Word Count":name_word_count,
            "Link":links_,

            }
    
    df = pd.DataFrame(data)
    return df

In [17]:
driver = webdriver.Chrome()
raw_data = get_data_into_df(link_list[0],driver)


#print(len(link_list))

for k in range(1,len(link_list)):
    temp_df = get_data_into_df(link_list[k],driver)
    raw_data = pd.concat([raw_data,temp_df])
    
    clear_output(wait=False) #clears the output
    print('{} out of {}'.format(k,len(link_list)-1)) #prints completion status
    
    if (k % 25 == 0):
        print('waiting 4 sec cause k % 25') #prints that you have to wait
        time.sleep(4)

#del raw_data['Unnamed: 0'] #deletes the wierd 'Unnamed: 0' column that is basically filled only with 0s

raw_data

6859 out of 6859


,Name Romaji,Name Native,Name English,Synonyms,Format,Chapter Count,Volume Count,Status,Start Date,Start Year,...,Rating Rank,Rating Number,Popularity Rank,Popularity Number,Related 1 Type,Related 2 Type,Has Anime,Anime Release Date,Name Word Count,Link
0,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu e,ようこそ実力至上主義の教室へ,Classroom of the Elite,Youjitsu\nYou-Zitsu\nขอต้อนรับสู่ห้องเรียนนิยม...,Light Novel,110,14,Finished,"May 25, 2015",2015.0,...,#3 Highest Rated All Time,3.0,#1 Most Popular All Time,1.0,TV · Finished,Manga · Releasing,True,2017.0,7.0,https://anilist.co/manga/94970/Youkoso-Jitsury...
0,Mikkakan no Koufuku,三日間の幸福,Three Days of Happiness,Ba ngày hạnh phúc\n三日間的幸福\nIch habe mein Leben...,Light Novel,15,1,Finished,"Dec 25, 2013",2013.0,...,#1 Highest Rated All Time,1.0,#2 Most Popular All Time,2.0,Manga · Finished,NaN,False,NaN,3.0,https://anilist.co/manga/98361/Mikkakan-no-Kou...
0,Mushoku Tensei: Isekai Ittara Honki Dasu,無職転生 ～異世界行ったら本気だす～,Mushoku Tensei: Jobless Reincarnation,เกิดชาตินี้พี่ต้องเทพ,Light Novel,NaN,26,Finished,"Jan 23, 2014",2014.0,...,#6 Highest Rated All Time,6.0,#3 Most Popular All Time,3.0,Manga · Releasing,TV · Finished,True,2021.0,6.0,https://anilist.co/manga/85470/Mushoku-Tensei-...
0,Utsuro no Hako to Zero no Maria,空ろの箱と零のマリア,The Empty Box and Zeroth Maria,Hakomari\nZerowa Maria i puste pudełko\nПустая...,Light Novel,254,7,Finished,"Jan 7, 2009",2009.0,...,#15 Highest Rated All Time,15.0,#4 Most Popular All Time,4.0,NaN,NaN,False,NaN,7.0,https://anilist.co/manga/85134/Utsuro-no-Hako-...
0,Re:Zero kara Hajimeru Isekai Seikatsu,Re:ゼロから始める異世界生活,Re:ZERO -Starting Life in Another World-,ReZero\nRe: Life in a different world from zer...,Light Novel,NaN,NaN,Releasing,"Jan 24, 2014",2014.0,...,#12 Highest Rated All Time,12.0,#5 Most Popular All Time,5.0,Manga · Finished,Manga · Finished,True,2016.0,5.0,https://anilist.co/manga/85737/ReZero-kara-Haj...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Kusuriya no Seijou: Kazoku ni Shiitagerareteit...,薬屋の聖女 ～家族に虐げられていた薬屋の女の子、実は世界一のポーションを作れるそうですよ～,NaN,NaN,Light Novel,NaN,NaN,Releasing,"Dec 2, 2022",2022.0,...,NaN,NaN,NaN,NaN,Manga · Releasing,NaN,False,NaN,19.0,https://anilist.co/manga/160126/Kusuriya-no-Se...
0,"Meikyuu Shokudou ""Maoukutsu"" e Youkoso! Tensei...",迷宮食堂『魔王窟』へようこそ！～転生してから300年も寝ていたので、飲食店経営で魔王を目指そ...,NaN,NaN,Light Novel,NaN,NaN,Releasing,"Apr 19, 2022",2022.0,...,NaN,NaN,NaN,NaN,Manga · Releasing,NaN,False,NaN,20.0,https://anilist.co/manga/160363/Meikyuu-Shokud...
0,Reide Fusai no Nare Some: Mayoikonda Kimi no H...,レイデ夫妻のなれそめ 迷い込んだ君の秘密,NaN,NaN,Light Novel,6,1,Finished,"Feb 15, 2016",2016.0,...,NaN,NaN,NaN,NaN,Light Novel · Finished,Light Novel · Finished,False,NaN,9.0,https://anilist.co/manga/149469/Reide-Fusai-no...
0,Jigen Bakuju,時限爆呪,NaN,NaN,Light Novel,NaN,1,Finished,"Dec 3, 1999",1999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,2.0,https://anilist.co/manga/150423/Jigen-Bakuju/


In [18]:
raw_data.to_csv('raw_data.csv')
raw_data.to_excel('raw_data_exc.xlsx')

In [19]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6860 entries, 0 to 0
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name Romaji         6471 non-null   object 
 1   Name Native         6463 non-null   object 
 2   Name English        601 non-null    object 
 3   Synonyms            2906 non-null   object 
 4   Format              6472 non-null   object 
 5   Chapter Count       2519 non-null   object 
 6   Volume Count        4060 non-null   object 
 7   Status              6471 non-null   object 
 8   Start Date          6434 non-null   object 
 9   Start Year          6434 non-null   float64
 10  End Date            1966 non-null   object 
 11  Average Score       345 non-null    object 
 12  Mean Score          3233 non-null   object 
 13  Popularity          6468 non-null   object 
 14  Favorites           2936 non-null   object 
 15  Source              6073 non-null   object 
 16  Genre_One

In [ ]:
dat = pd.read_csv("raw_data.csv")
dat

In [23]:
driver = webdriver.Chrome()
new_data = get_data_into_df(link_list[0],driver)


#print(len(link_list))

for k in range(1,650):
    temp_df = get_data_into_df(link_list[k],driver)
    new_data = pd.concat([new_data,temp_df])
    
    clear_output(wait=False) #clears the output
    print('{} out of {}'.format(k,len(link_list)-1)) #prints completion status
    
    if (k % 10 == 0):
        print('waiting 10 sec cause k % 10') #prints that you have to wait
        time.sleep(10)

#del raw_data['Unnamed: 0'] #deletes the wierd 'Unnamed: 0' column that is basically filled only with 0s

new_data

649 out of 6859


,Name Romaji,Name Native,Name English,Synonyms,Format,Chapter Count,Volume Count,Status,Start Date,Start Year,...,Rating Rank,Rating Number,Popularity Rank,Popularity Number,Related 1 Type,Related 2 Type,Has Anime,Anime Release Date,Name Word Count,Link
0,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu e,ようこそ実力至上主義の教室へ,Classroom of the Elite,Youjitsu\nYou-Zitsu\nขอต้อนรับสู่ห้องเรียนนิยม...,Light Novel,110,14,Finished,"May 25, 2015",2015,...,#3 Highest Rated All Time,3.0,#1 Most Popular All Time,1.0,TV · Finished,Manga · Releasing,True,2017.0,7,https://anilist.co/manga/94970/Youkoso-Jitsury...
0,Mikkakan no Koufuku,三日間の幸福,Three Days of Happiness,Ba ngày hạnh phúc\n三日間的幸福\nIch habe mein Leben...,Light Novel,15,1,Finished,"Dec 25, 2013",2013,...,#1 Highest Rated All Time,1.0,#2 Most Popular All Time,2.0,Manga · Finished,NaN,False,NaN,3,https://anilist.co/manga/98361/Mikkakan-no-Kou...
0,Mushoku Tensei: Isekai Ittara Honki Dasu,無職転生 ～異世界行ったら本気だす～,Mushoku Tensei: Jobless Reincarnation,เกิดชาตินี้พี่ต้องเทพ,Light Novel,NaN,26,Finished,"Jan 23, 2014",2014,...,#6 Highest Rated All Time,6.0,#3 Most Popular All Time,3.0,Manga · Releasing,TV · Finished,True,2021.0,6,https://anilist.co/manga/85470/Mushoku-Tensei-...
0,Utsuro no Hako to Zero no Maria,空ろの箱と零のマリア,The Empty Box and Zeroth Maria,Hakomari\nZerowa Maria i puste pudełko\nПустая...,Light Novel,254,7,Finished,"Jan 7, 2009",2009,...,#15 Highest Rated All Time,15.0,#4 Most Popular All Time,4.0,NaN,NaN,False,NaN,7,https://anilist.co/manga/85134/Utsuro-no-Hako-...
0,Re:Zero kara Hajimeru Isekai Seikatsu,Re:ゼロから始める異世界生活,Re:ZERO -Starting Life in Another World-,ReZero\nRe: Life in a different world from zer...,Light Novel,NaN,NaN,Releasing,"Jan 24, 2014",2014,...,#12 Highest Rated All Time,12.0,#5 Most Popular All Time,5.0,Manga · Finished,Manga · Finished,True,2016.0,5,https://anilist.co/manga/85737/ReZero-kara-Haj...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Shuuen no Shiori,終焉ノ栞,NaN,Bookmark of Demise,Light Novel,23,4,Finished,"Feb 25, 2013",2013,...,#29 Highest Rated 2013,29.0,NaN,NaN,Manga · Finished,Manga · Finished,False,NaN,3,https://anilist.co/manga/79675/Shuuen-no-Shiori/
0,Toaru Hikuushi e no Tsuioku,とある飛空士への追憶,NaN,Toaru Hikoushi eno Tsuioku\nA Certain Recollec...,Light Novel,13,1,Finished,"Feb 19, 2008",2008,...,#23 Highest Rated 2008,23.0,#23 Most Popular 2008,23.0,Movie · Finished,Manga · Finished,True,2011.0,5,https://anilist.co/manga/47285/Toaru-Hikuushi-...
0,Yuusha ni Narenakatta Ore wa Shibushibu Shuush...,勇者になれなかった俺はしぶしぶ就職を決意しました。,NaN,Yuusha ni Narenakatta Ore wa Shibushibu Shuush...,Light Novel,77,10,Finished,"Jan 20, 2012",2012,...,NaN,NaN,NaN,NaN,TV · Finished,Manga · Finished,True,2013.0,10,https://anilist.co/manga/78961/Yuusha-ni-Naren...
0,"Ore, Twintail ni Narimasu.",俺、ツインテールになります。,NaN,Gonna be the Twin-Tail!!\nOre\nTwin tails ni N...,Light Novel,NaN,NaN,Releasing,"Jun 19, 2012",2012,...,NaN,NaN,NaN,NaN,TV · Finished,Light Novel · Finished,True,2014.0,4,https://anilist.co/manga/88187/Ore-Twintail-ni...


In [24]:
new_data.to_csv('data_firsts.csv')
new_data.to_excel('data_firsts.xlsx')
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 650 entries, 0 to 0
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name Romaji         650 non-null    object 
 1   Name Native         649 non-null    object 
 2   Name English        327 non-null    object 
 3   Synonyms            538 non-null    object 
 4   Format              650 non-null    object 
 5   Chapter Count       363 non-null    object 
 6   Volume Count        436 non-null    object 
 7   Status              650 non-null    object 
 8   Start Date          650 non-null    object 
 9   Start Year          650 non-null    int64  
 10  End Date            267 non-null    object 
 11  Average Score       534 non-null    object 
 12  Mean Score          649 non-null    object 
 13  Popularity          650 non-null    object 
 14  Favorites           649 non-null    object 
 15  Source              597 non-null    object 
 16  Genre_One 

In [25]:
dv = webdriver.Chrome()

ik = get_data_into_df("https://anilist.co/manga/85134/Utsuro-no-Hako-to-Zero-no-Maria/",dv)
ik

,Name Romaji,Name Native,Name English,Synonyms,Format,Chapter Count,Volume Count,Status,Start Date,Start Year,...,Rating Rank,Rating Number,Popularity Rank,Popularity Number,Related 1 Type,Related 2 Type,Has Anime,Anime Release Date,Name Word Count,Link
0,Utsuro no Hako to Zero no Maria,空ろの箱と零のマリア,The Empty Box and Zeroth Maria,Hakomari\nZerowa Maria i puste pudełko\nПустая...,Light Novel,254,7,Finished,"Jan 7, 2009",2009,...,#15 Highest Rated All Time,15,#4 Most Popular All Time,4,NaN,NaN,False,NaN,7,https://anilist.co/manga/85134/Utsuro-no-Hako-...
